In [1]:
import sys
sys.path.append('../')

In [2]:
import CS_Indicators as CS
from brix import Indicator, Handler
import PreCompOsmNet
import Simulation
import geopandas as gpd
import pandas as pd
import requests

In [3]:
# import sklearn

In [4]:
table_name='epaversion'

In [5]:
##lUIS SILVA CÓDIGO
r = requests.get('https://cityscope-api.smartaraucania.org/api/table/epa/GEOGRID')


### Load the Saved Data

In [6]:
geogrid=gpd.read_file('../tables/{}/geogrid.geojson'.format(table_name))
#zones=gpd.read_file(r.text, driver ='GeoJSON')
zones=gpd.read_file('../tables/{}/zones.geojson'.format(table_name))
zones['GEOID']=zones['GEOID'].astype('int64')
zones=zones.set_index('GEOID')
simpop_df=pd.read_csv('../tables/{}/simpop_df.csv'.format(table_name))

### LODES industry is not useful- look up NAICS from wac instead
- TODO: do this in the script which prepares the USA data

In [7]:
import random
naics_cols=[col for col in zones.columns if 'naics' in col]
simpop_df['naics']=0
for ind, row in simpop_df.iterrows():
    if zones.loc[row['work_geoid'], 'emp_total']>0:
        simpop_df.loc[ind, 'naics']=random.choices(naics_cols, weights=zones.loc[row['work_geoid'], naics_cols].values, k=1)[0]
    else:
        simpop_df.loc[ind, 'naics']=random.choices(naics_cols, k=1)[0]

In [8]:
simpop_df=simpop_df.drop('industry', axis=1)

### Create the mobility system

In [9]:
networks, mode_dicts, route_lengths=PreCompOsmNet.create_2_scale_osmnx_network(
    zones.loc[zones['sim_area']], zones.loc[zones['model_area']],
    add_modes=[{'name': 'walk', 'speed': 4800/3600},
               {'name': 'cycle', 'speed': 14000/3600},
               {'name': 'pt', 'speed': 25000/3600}])
modes={mode: Simulation.Mode(mode_dicts[mode]) for mode in mode_dicts}
mob_sys=Simulation.MobilitySystem(modes=modes, networks=networks)

getting internal roads
Buffer : 200
getting external roads
	 Simplified from 1404 to 967 edges and 950 to 604 nodes
	 Pre-computing paths


In [10]:
import numpy as np

class Default_Mode_Choice_model():
    def __init__(self):
        pass
    
    def predict_modes(self, all_trips_df):
        options=['drive', 'cycle', 'walk', 'pt']
        all_trips_df['mode']='drive'
        ind_u1500=all_trips_df['route_distance']<1500
        ind_1500_5k=((all_trips_df['route_distance']>1500)&(all_trips_df['route_distance']<5000))
        ind_5k_10k=((all_trips_df['route_distance']>5000)&(all_trips_df['route_distance']<10000))
        ind_10kplus=all_trips_df['route_distance']>10000

        all_trips_df.loc[ind_u1500==True, 'mode']=np.random.choice(
            options, size=sum(ind_u1500), 
            replace=True, p=[0.1, 0.2, 0.6, 0.1])

        all_trips_df.loc[ind_1500_5k==True, 'mode']=np.random.choice(
            options, size=sum(ind_1500_5k), 
            replace=True, p=[0.25, 0.15, 0.3, 0.3])

        all_trips_df.loc[ind_5k_10k==True, 'mode']=np.random.choice(
            options, size=sum(ind_5k_10k), 
            replace=True, p=[0.6, 0.04, 0.01, 0.35])

        all_trips_df.loc[ind_10kplus==True, 'mode']=np.random.choice(
            options, size=sum(ind_10kplus), 
            replace=True, p=[0.9, 0.01, 0.01, 0.08])
        return all_trips_df

mode_descriptions = [{"name": 'drive',
                    'color': "#e41a1d"},
                     {"name": 'cycle',
                    'color': "#377eb8"},
                     {"name": 'walk',
                    'color': "#4daf4a"},
                     {"name": 'pt',
                    'color': "#ffff33"},
                    ]

### Create and run the Cityscope model

In [11]:
H=Handler(table_name=table_name)
H.reset_geogrid_data()

https://cityscope-api.smartaraucania.org/api/tables/list/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/GEOGRID/ Attempt: 0
Proceeding without reviewing GEOGRIDDATA
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
GEOGRIDDATA successfully updated: wTFjHvG8e6CSSBeuuwjArjh2fcP5GUNMpwkeTgRWdif


In [12]:
d=CS.Density_Indicator(zones=zones)
#p=CS.Proximity_Indicator(zones=zones, geogrid=geogrid)
m=CS.Mobility_indicator(zones, geogrid, table_name, simpop_df, mob_sys,
                       mode_choice_model=Default_Mode_Choice_model(),
                       mode_descriptions=mode_descriptions,
                       route_lengths=route_lengths)

H.add_indicators([
    d, 
    #p, 
    m
])

# H.add_indicator(m)

Init simulation


c:\Users\daira\OneDrive\Escritorio\Spacial-Modules\CS_Spatial_Modules\examples\..\CS_Indicators.py:625: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids=grid_zones['geometry'].centroid
c:\Users\daira\OneDrive\Escritorio\Spacial-Modules\CS_Spatial_Modules\examples\..\CS_Indicators.py:593: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_zones=model_zones.append(grid_zones)


Getting internal nodes
Finding closest nodes to every zone centroid
Computing zone-zone dist mat
https://cityscope-api.smartaraucania.org/api/table/epaversion/GEOGRIDDATA/ Attempt: 0


c:\Users\daira\miniconda3\envs\SpacialModules\lib\site-packages\brix\classes.py:982: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  geos = gpd.GeoDataFrame(geos.drop('geometry',1),geometry=geos['geometry'].apply(lambda x: shape(x))) # no crs to avoid warning


Dens Ind: 0:00:00.003007
https://cityscope-api.smartaraucania.org/api/table/epaversion/GEOGRIDDATA/ Attempt: 0
Starting MM Update
Schedules and locations
OD
Trip table
Route table
DeckGL


c:\Users\daira\OneDrive\Escritorio\Spacial-Modules\CS_Spatial_Modules\examples\..\CS_Indicators.py:753: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_simpop=self.base_simpop_df.append(new_simpop_df)
c:\Users\daira\OneDrive\Escritorio\Spacial-Modules\CS_Spatial_Modules\examples\..\Simulation.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
c:\Users\daira\OneDrive\Escritorio\Spacial-Modules\CS_Spatial_Modules\examples\..\Simulation.py:408: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

Post ABM: <Response [404]>
Post OD: <Response [404]>
Finished MM Update
	 calculation took 0:00:00.224648
	 trips post took 0:00:06.032861


In [13]:
# self=m
# import datetime
# print('Starting MM Update')
# start_calc=datetime.datetime.now()
# new_simpop=self.geogrid_updates(geogrid_data)
# new_simpop_df=pd.DataFrame(new_simpop)
# combined_simpop=self.base_simpop_df.append(new_simpop_df)
# sample_simpop_df=combined_simpop.sample(min(self.N_max, len(combined_simpop)))
# od_output, deckgl_trips, ind=self.simulate(sample_simpop_df)

In [14]:
geogrid_data=H.get_geogrid_data()
print(m.return_indicator(geogrid_data))
#print(geogrid_data)
#print(p.return_indicator(geogrid_data))
#print(d.return_indicator(geogrid_data))

https://cityscope-api.smartaraucania.org/api/table/epaversion/GEOGRIDDATA/ Attempt: 0
Starting MM Update
Schedules and locations
OD
Trip table
Route table
DeckGL


c:\Users\daira\OneDrive\Escritorio\Spacial-Modules\CS_Spatial_Modules\examples\..\CS_Indicators.py:753: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_simpop=self.base_simpop_df.append(new_simpop_df)
c:\Users\daira\OneDrive\Escritorio\Spacial-Modules\CS_Spatial_Modules\examples\..\Simulation.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
c:\Users\daira\OneDrive\Escritorio\Spacial-Modules\CS_Spatial_Modules\examples\..\Simulation.py:408: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

Post ABM: <Response [404]>
Post OD: <Response [404]>
Finished MM Update
	 calculation took 0:00:00.327991
	 trips post took 0:00:06.220343
{'name': 'Active Mobility', 'raw_value': 0.092, 'value': 0.092, 'ref_value': 0.1, 'viz_type': 'radar'}


In [15]:
H.listen()

Table URL: https://cityscope-api.smartaraucania.org/CS_cityscopeJS/?cityscope=epaversion
Testing indicators
https://cityscope-api.smartaraucania.org/api/table/epaversion/GEOGRIDDATA/ Attempt: 0
Dens Ind: 0:00:00.003994
Starting MM Update
Schedules and locations
OD
Trip table
Route table
DeckGL


c:\Users\daira\OneDrive\Escritorio\Spacial-Modules\CS_Spatial_Modules\examples\..\CS_Indicators.py:753: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_simpop=self.base_simpop_df.append(new_simpop_df)
c:\Users\daira\OneDrive\Escritorio\Spacial-Modules\CS_Spatial_Modules\examples\..\Simulation.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
c:\Users\daira\OneDrive\Escritorio\Spacial-Modules\CS_Spatial_Modules\examples\..\Simulation.py:408: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

Post ABM: <Response [404]>
Post OD: <Response [404]>
Finished MM Update
	 calculation took 0:00:00.277448
	 trips post took 0:00:12.159591
Performing initial update
Updating geogrid_data
https://cityscope-api.smartaraucania.org/api/table/epaversion/GEOGRIDDATA/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
Updating table with hash: wTFjHvG8e6CSSBeuuwjArjh2fcP5GUNMpwkeTgRWdif
https://cityscope-api.smartaraucania.org/api/table/epaversion/GEOGRIDDATA/ Attempt: 0
Dens Ind: 0:00:00.005000
Starting MM Update
Schedules and locations
OD
Trip table
Route table
DeckGL


c:\Users\daira\OneDrive\Escritorio\Spacial-Modules\CS_Spatial_Modules\examples\..\CS_Indicators.py:753: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_simpop=self.base_simpop_df.append(new_simpop_df)
c:\Users\daira\OneDrive\Escritorio\Spacial-Modules\CS_Spatial_Modules\examples\..\Simulation.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
c:\Users\daira\OneDrive\Escritorio\Spacial-Modules\CS_Spatial_Modules\examples\..\Simulation.py:408: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

Post ABM: <Response [404]>
Post OD: <Response [404]>
Finished MM Update
	 calculation took 0:00:00.413813
	 trips post took 0:00:02.374288
Done with update
Local grid hash: wTFjHvG8e6CSSBeuuwjArjh2fcP5GUNMpwkeTgRWdif
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://c

c:\Users\daira\OneDrive\Escritorio\Spacial-Modules\CS_Spatial_Modules\examples\..\CS_Indicators.py:753: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_simpop=self.base_simpop_df.append(new_simpop_df)
c:\Users\daira\OneDrive\Escritorio\Spacial-Modules\CS_Spatial_Modules\examples\..\Simulation.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
c:\Users\daira\OneDrive\Escritorio\Spacial-Modules\CS_Spatial_Modules\examples\..\Simulation.py:408: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

Route table
DeckGL
Post ABM: <Response [404]>
Post OD: <Response [404]>
Finished MM Update
	 calculation took 0:00:00.505103
	 trips post took 0:00:05.986305
Done with update
Local grid hash: B83pYnDqz2JPjd6CYapUuohc5VCMMXGgzXyzHWfSk2EN
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ 

c:\Users\daira\miniconda3\envs\SpacialModules\lib\site-packages\brix\classes.py:1104: UserWarning: Indicator not working:Mobility
  warn('Indicator not working:'+str(indicator_name))
c:\Users\daira\miniconda3\envs\SpacialModules\lib\site-packages\brix\classes.py:1105: UserWarning: Traceback (most recent call last):
  File "c:\Users\daira\miniconda3\envs\SpacialModules\lib\site-packages\brix\classes.py", line 1098, in update_package
    new_values_numeric += self._new_value(geogrid_data,indicator_name)
  File "c:\Users\daira\miniconda3\envs\SpacialModules\lib\site-packages\brix\classes.py", line 854, in _new_value
    new_value_raw = I.return_indicator(geogrid_data)
  File "c:\Users\daira\OneDrive\Escritorio\Spacial-Modules\CS_Spatial_Modules\examples\..\CS_Indicators.py", line 751, in return_indicator
    new_simpop=self.geogrid_updates(geogrid_data)
  File "c:\Users\daira\OneDrive\Escritorio\Spacial-Modules\CS_Spatial_Modules\examples\..\CS_Indicators.py", line 705, in geogrid_updates

Done with update
Local grid hash: C4vo4evs2Rk7UurZTXQnPndQAdorozdUBKmviN1gnJws
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion/meta/hashes/ Attempt: 0
https://cityscope-api.smartaraucania.org/api/table/epaversion

KeyboardInterrupt: 